In [14]:
seed_url = 'https://www.ku.ac.th/th/'
frontier_q = [seed_url]
visited_q = []
visited_netloc = []
success_html = []
domain = 'ku.ac.th'
EXCEPT_FILE = ('pdf', 'doc', 'xls', 'ppt', 'exe', 'jpg', 'mpg', 'zip')
WEB_TYPE = ("html", "htm", "asp", "php")
PAGE_COUNT = 0
DEBUG = False
LOG_WRITE = True
LOG_READ = True

headers = {
    'User-Agent': 'mayo_bot',
    'From': 'ittiwat.ma@ku.th'
}

DEBUG, READLOG, WRITELOG

In [15]:
def read_log(filename):
    f = open('log/'+filename, 'r',encoding='utf-8')
    return f.read()

def log_to_array(raw_file):
    return raw_file.split('\n')

def write_txt_log(list_string, filename, type = "list"):
    os.makedirs("log", 0o755, exist_ok=True)
    if type == "list":
        list_string = "\n".join(list_string)
    f = codecs.open("log/"+filename+".txt", 'w', 'utf-8')
    f.write(list_string)
    f.close()
    
def write_log_file(f = 250):
    if(PAGE_COUNT%f == 0):
        write_txt_log(visited_netloc, "visited_netloc",)
        write_txt_log(visited_q, "visited_q")
        write_txt_log(success_html, "success_html")
        write_txt_log(frontier_q, "frontier_q")
        write_txt_log(str(PAGE_COUNT), "PAGE_COUNT", "number")

def debug(url,type):
    global PAGE_COUNT
    try:
        if('text/html' in type):
            print('page: ',PAGE_COUNT)
            print('site: ',url)
            print(type)
            print('Write success')
            print('-'*50)
        else:
            print('site: ',url)
            print(type)
            print('-'*50)
    except:
        pass
    
if LOG_READ:
    try:
        frontier_q = log_to_array(read_log('frontier_q.txt'))
        visited_q = log_to_array(read_log('visited_q.txt'))
        visited_netloc = log_to_array(read_log('visited_netloc.txt'))
        success_html = log_to_array(read_log('success_html.txt'))
        PAGE_COUNT = int(read_log('PAGE_COUNT.txt'))
    except:
        print('no log')


WRITE FILE HTML

In [16]:
import os, codecs
from urllib.parse import urljoin, urlparse, unquote


def write_html_with_path(url, raw_html):
    path = ''
    file_type = ''
    abs_file = ''
    link = urlparse(url)
    netloc = link.netloc
    path = link.path
    
    if(path.endswith(WEB_TYPE)):
        file_path = 'html/' + netloc + '/'.join(path.split("/")[:-1])+ "/"
        abs_file = file_path  + path.split("/")[-1]
    else:
        file_path = 'html/' + netloc + path + '/'
        abs_file = file_path  + "dummy"
    
    os.makedirs(file_path, 0o755, exist_ok=True)
    f = codecs.open(abs_file, 'w', 'utf-8')
    f.write(raw_html)
    f.close()

FUNCTION

In [17]:
import requests
from requests.exceptions import HTTPError

#get page html 
def get_page(url, robots = False):
    global headers
    global PAGE_COUNT
    text = ''
    try:
        response = requests.get(url, headers=headers, timeout=2)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        if(DEBUG):
            print("fail!")
            print(f'HTTP error occurred: {http_err}')  # Python 3.6
            print('-'*50)
    except Exception as err:
        if(DEBUG):
            print("fail!")
            print(f'Other error occurred: {err}')  # Python 3.6
            print('-'*50)
    else:
        try:
            if(robots == False):
                if('text/html; charset=UTF-8' == response.headers['Content-Type'] ):
                    PAGE_COUNT += 1
                    success_html.append(url)
                    response.encoding = 'utf-8'
                    text = response.text    
                    write_html_with_path(url,text)
                    
                    #visit netloc history
                    link  = urlparse(url)
                    link_netlock = link.scheme + "://" + link.netloc
                    if link_netlock not in visited_netloc:
                        visited_netloc.append(link_netlock)
                if(DEBUG):
                    debug(url,response.headers['Content-Type'])
                if(LOG_WRITE):
                    write_log_file()
            if(robots):
                if('text/plain' in response.headers['Content-Type']):
                    text = response.text    
        except:
            pass
    return text.lower()

In [18]:
#find a href link in current url
def get_all_link_form_url(current_url):
    urls = []
    raw_html = get_page(current_url)
    pattern_start = '<a href="';  pattern_end = '"'
    index = 0;  length = len(raw_html)
    while index < length:
        start = raw_html.find(pattern_start, index)
        if start > 0:
            start = start + len(pattern_start)
            end = raw_html.find(pattern_end, start)
            link = raw_html[start:end]
            if len(link) > 0:
                if link not in urls:
                    link = urljoin(current_url, link)
                    url = urlparse(link)
                    link = url.scheme + '://' + url.netloc + url.path
                    urls.append(link)
            index = end
        else:
            break
    
    
    return urls

In [19]:

def enqueue(links):
    global frontier_q
    for link in links:
        if link not in frontier_q and link not in visited_q and domain in link and not link.endswith(EXCEPT_FILE) and 'mailto:/' not in link:
            frontier_q.append(link)

# FIFO queue
def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

MAIN

In [20]:
#main
while (PAGE_COUNT < 10001 and frontier_q):
    current_url = dequeue()
    current_url = unquote(current_url.strip())
    visited_q.append(current_url)
    extracted_links = get_all_link_form_url(current_url)
    enqueue(extracted_links)
    
    
    
write_log_file(1)
print("="*55)
print("DONE !"*9)
print("="*55)


DONE !DONE !DONE !DONE !DONE !DONE !DONE !DONE !DONE !


ROBOTS, SITEMAP

In [21]:
robots_web = []
sitemap_web = []

for site in visited_netloc:
    t = get_page(site + '/robots.txt', True)
    if(t):
        robots_web.append('https://'+ urlparse(site).netloc)
        if('sitemap:' in t):
            sitemap_web.append('https://'+urlparse(site).netloc)

f = codecs.open('list_robots.txt', 'w', 'utf-8')
f.write('\n'.join(robots_web))
f.close()

f = codecs.open('list_sitemap.txt', 'w', 'utf-8')
f.write('\n'.join(sitemap_web))
f.close()


KeyboardInterrupt: 